In [207]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta

In [208]:
# path to your service account key JSON file
key_file_path = "./smartwaiver-388021-9a8ea3b56852.json"

# Use your JSON key to authenticate your account
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_path, scope)

client = gspread.authorize(creds)


In [209]:
# Open the Google Spreadsheet by its name (make sure you've shared it with the client_email from your service account)
sheet = client.open('smartwaiver_test').sheet1

In [210]:
# put the full file name for the smartwaiver data here
smartwaiver = 'smartwaiver-173467-647a0861807c9.csv'

# load the data from your CSV file
data = pd.read_csv(smartwaiver)

In [211]:
# convert 'Waiver Date' column to datetime
data['Date Completed (UTC)'] = pd.to_datetime(data['Date Completed (UTC)'])

In [212]:
# get the current date
now = datetime.now()

# calculate the date one year ago
one_year_ago = now - timedelta(days=365)

In [213]:
# filter the data to get volunteers who signed a waiver in the past year
filtered_data = data[data['Date Completed (UTC)'] >= one_year_ago]

In [214]:
filtered_data.head(5)

,First,Middle,Last,Date of Birth,Date Completed (UTC),Waiver ID,Status,Check-ins,Gender,Phone,...,Parent/Guardian First Name,Parent/Guardian Middle Name,Parent/Guardian Last Name,Parent/Guardian Phone,Parent/Guardian Relationship,Email,signing_type,template_type,Title of Document,typed_font
0,Patrick,NaN,Hayes,12/11/2007,2023-04-26 14:58:52,fgEHFCkdNYZJJMuTgY8BzU,Completed Online,0,NaN,NaN,...,Amy,NaN,Hayes,NaN,NaN,aabigailh@gmail.com,draw,1,SCMTS Waiver,dancing
1,Mark,NaN,Phillips,NaN,2023-04-26 15:03:37,EFyeg52WYcWnW9wj7VjfZ8,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,mark.d.phillips.2555@gmail.com,draw,1,SCMTS Waiver,dancing
2,Peter,NaN,Nell,NaN,2023-04-26 15:05:28,Mhxdw3q6rrhNVigGQbnH8E,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,peteralbertnell@gmail.com,draw,1,SCMTS Waiver,dancing
3,Raven,NaN,Adams,NaN,2023-04-26 15:06:14,KjmejHvvZEfNoQRJUnbsz6,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ravenadams08@gmail.com,draw,1,SCMTS Waiver,dancing
4,Elise,NaN,Roy,NaN,2023-04-26 15:08:17,P4pDstc74pMuYuhV8orcoB,Completed Online,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,elise.c.roy@gmail.com,draw,1,SCMTS Waiver,dancing


In [215]:
# Get all records from the Google Sheet
records = sheet.get_all_records()

# Create an empty DataFrame to store unmatched names
unmatched_names = pd.DataFrame(columns=['First Name', 'Last Name', 'Preferred Email'])

In [216]:

# get the list of column names
headers = sheet.row_values(1)

# find the index of the 'SCMTS Waiver' column (Python index starts at 0 so we need to add 1 to match gspread formatting)
waiver_column_index = headers.index('SCMTS Waiver') + 1

# iterate over the filtered data
for index, row in filtered_data.iterrows():
    
    # find the volunteer in the Google Spreadsheet
    matched_records = [record for record in records if record['Contact Name'].split(", ")[1] == row['First'] and record['Contact Name'].split(", ")[0] == row['Last']]

    if matched_records:
          
        # if there is a match check the box
        record = matched_records[0]
        
        row_number = records.index(record) + 2 # Adds 2 because gspread row index starts at 1 and we have a header row

        # check the box in the 4th column
        sheet.update_cell(row_number, 4, True) # The number 3 reflects the column number

In [217]:
# iterate over the records from the Google Sheets
for record in records:
    # check if the box is still 'False'
    if record['SCMTS Waiver'] == 'FALSE':  # gspread recognizes empty checkboxes as a string 'FALSE'
        # if it's still 'False', add the contact name and email to the DataFrame
        name_parts = record['Contact Name'].split(", ")
        if len(name_parts) == 2: # Making sure the name is in "Last, First" format
            unmatched_names = unmatched_names.append({'First': name_parts[1], 'Last': name_parts[0], 'Email': record['Preferred Email']}, ignore_index=True)

In [218]:
unmatched_names

,First Name,Last Name,Preferred Email,Email,First,Last
0,NaN,NaN,NaN,cara.wetlandswatch@gmail.com,Cara,Clark
1,NaN,NaN,NaN,Babyblue_bs@hotmail.com,Fabiola,DeAguinaga
2,NaN,NaN,NaN,agile24@live.com,Alyss,Gile
3,NaN,NaN,NaN,Jessica.m.hogan91@gmail.com,Jessica,Hogan
4,NaN,NaN,NaN,missperla17@me.com,perla,leal
5,NaN,NaN,NaN,nancy.porto@watsonville.gov,Nancy,Porto
6,NaN,NaN,NaN,mariano@santacruztrails.org,Mariano,Villegas
7,NaN,NaN,NaN,catherinehweb@gmail.com,Catherine,Weber
